In [1]:
import pandas as pd
import numpy as np

In [2]:
people_df = pd.read_csv(r"C:\Users\19692\Downloads\CSE560-Project\title.principals.tsv", sep="\t")
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91762951 entries, 0 to 91762950
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   tconst      object
 1   ordering    int64 
 2   nconst      object
 3   category    object
 4   job         object
 5   characters  object
dtypes: int64(1), object(5)
memory usage: 4.1+ GB


## Process for Casts

In [3]:
casts_df = people_df[people_df['characters']!='\\N']
casts_df = casts_df[['tconst','nconst','characters']]

In [4]:
# process characters column
# ["superman, DC"] --> superman/DC
def clean_characters(val):
    if isinstance(val, str):
        return val.strip("[]").replace('"', '').replace("'", '').replace(',','/').replace(';','/')
    return val

casts_df['characters'] = casts_df['characters'].apply(clean_characters)


In [5]:
# drop duplicate
casts_df = casts_df.drop_duplicates()

## Process for Other Professionals

In [6]:
people_df = people_df[people_df['characters']=='\\N']
people_df = people_df[['tconst','nconst','category','job']]


In [ ]:
# if job is /N, make it be the value of category column
people_df.loc[people_df['job'] == '\\N', 'job'] = people_df['category']


In [32]:
# get job people
job_people = people_df[['tconst','nconst','job']].drop_duplicates().reset_index(drop=True)
job_people['job'] = job_people['job'].str.replace(',', '/', regex=False).str.replace(';', '/', regex=False)

In [33]:
job_category = people_df[['job', 'category']].drop_duplicates().reset_index(drop=True)


In [34]:
job_category['job'] = job_category['job'].str.replace(',', '/', regex=False).str.replace(';', '/', regex=False)



## Connect to PostgreSQL

In [54]:
import psycopg2
import io

conn = psycopg2.connect(
    dbname="postgres",
    user="postgres.jzysaequuaturayaiczd",
    password="12345jiabaoyaogary",
    host="aws-0-us-west-1.pooler.supabase.com",
    port=6543
)
cur = conn.cursor()

In [58]:
conn.rollback()

In [55]:
cur.execute("SET statement_timeout = '600000';")  # 10 minutes in mscur.execute("SET statement_timeout = 0;")

In [47]:
conn.close()

## Upload Casts Relation

In [ ]:
"""import tempfile

with tempfile.NamedTemporaryFile(mode='w+', delete=False) as f:
    casts_df.to_csv(f.name, index=False, header=False)
    f.seek(0)
    
    # Then upload using COPY
    with open(f.name, 'r') as file_obj:
        cur.copy_from(file_obj, 'Cast', sep=',', null='')

conn.commit()
"""

"import tempfile\n\nwith tempfile.NamedTemporaryFile(mode='w+', delete=False) as f:\n    casts_df.to_csv(f.name, index=False, header=False)\n    f.seek(0)\n    \n    # Then upload using COPY\n    with open(f.name, 'r') as file_obj:\n        cur.copy_from(file_obj, 'Cast', sep=',', null='')\n\nconn.commit()\n"

## Upload Principal

In [51]:
# Step 1: Create a CSV buffer from the DataFrame
buffer = io.StringIO()
job_people.to_csv(buffer, index=False, header=False)
buffer.seek(0)

buffer1 = io.StringIO()
job_category.to_csv(buffer1, index=False, header=False)
buffer1.seek(0)


0

In [60]:
# upload data to job_people relation
cur.copy_from(buffer, 'job_people', sep=',', null='')
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


47307741

In [59]:
# upload data to job_category relation
cur.copy_from(buffer1, 'job_category', sep=',', null='')
conn.commit()

ReadOnlySqlTransaction: cannot execute COPY FROM in a read-only transaction
